In [1]:
import tensorflow as tf
import numpy as np
from pandas import read_csv, DataFrame
from scipy import stats
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# CNN Classification

In [34]:
features_df = read_csv('/content/drive/My Drive/Colab Notebooks/Kaggle Competition/train_features.csv')
labels_df = read_csv('/content/drive/My Drive/Colab Notebooks/Kaggle Competition/train_labels.csv')
features = features_df.values[:,1]
for i in range(len(features)):
  temp = np.zeros(600)
  for j in range(len(temp)):
    try:
      if features[i][j] == 'A':
        temp[j] = 1
      elif features[i][j] == 'C':
        temp[j] = 2
      elif features[i][j] == 'G':
        temp[j] = 3
      elif features[i][j] == 'T':
        temp[j] = 4
      else:
        temp[j] = 0
    except IndexError:
      temp[j] = 0
  features[i] = temp
  features[i] = tf.keras.utils.to_categorical(features[i])
features = np.stack(features)
labels = labels_df.values
trainy = tf.keras.utils.to_categorical(labels[:,1])

test_features_df = read_csv('/content/drive/My Drive/Colab Notebooks/Kaggle Competition/test_features.csv')
test_features = test_features_df.values[:,1]
for i in range(len(test_features)):
  temp = np.zeros(600)
  for j in range(len(temp)):
    try:
      if test_features[i][j] == 'A':
        temp[j] = 1
      elif test_features[i][j] == 'C':
        temp[j] = 2
      elif test_features[i][j] == 'G':
        temp[j] = 3
      elif test_features[i][j] == 'T':
        temp[j] = 4
      else:
        temp[j] = 0
    except IndexError:
      temp[j] = 0
  test_features[i] = temp
  test_features[i] = tf.keras.utils.to_categorical(test_features[i])
test_features = np.stack(test_features)

In [35]:
n_locations, n_features, n_outputs = features.shape[1], features.shape[2], trainy.shape[1]

In [36]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(10,13,activation='relu',input_shape=(n_locations, n_features)))#,kernel_regularizer=tf.keras.regularizers.l2(l2=.01)))
#model.add(tf.keras.layers.Dropout(.5))
model.add(tf.keras.layers.MaxPool1D(pool_size=3))
model.add(tf.keras.layers.Conv1D(20, 13, activation='relu'))#,kernel_regularizer=tf.keras.regularizers.l2(l2=.01)))
model.add(tf.keras.layers.MaxPool1D(pool_size=3))
model.add(tf.keras.layers.Conv1D(40, 13, activation='relu'))#,kernel_regularizer=tf.keras.regularizers.l2(l2=.01)))
model.add(tf.keras.layers.MaxPool1D(pool_size=3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(.25))
model.add(tf.keras.layers.Dense(75, activation='relu'))#,kernel_regularizer=tf.keras.regularizers.l1(l1=.01)))
model.add(tf.keras.layers.Dense(n_outputs,activation='softmax'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 588, 10)           660       
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 196, 10)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 184, 20)           2620      
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 61, 20)            0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 49, 40)            10440     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 16, 40)            0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 640)              

In [38]:
model.compile(optimizer='adam',
              loss ='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(features[11000:], trainy[11000:], validation_data=(features[11000:],trainy[11000:]), batch_size=64, epochs=5)

Epoch 1/5
202/202 [==============================] - 12s 54ms/step - loss: 0.0773 - accuracy: 0.9774 - val_loss: 0.0298 - val_accuracy: 0.9900
Epoch 2/5
202/202 [==============================] - 11s 52ms/step - loss: 0.0605 - accuracy: 0.9826 - val_loss: 0.0283 - val_accuracy: 0.9921
Epoch 3/5
202/202 [==============================] - 11s 53ms/step - loss: 0.0504 - accuracy: 0.9857 - val_loss: 0.0171 - val_accuracy: 0.9942
Epoch 4/5
202/202 [==============================] - 11s 53ms/step - loss: 0.0495 - accuracy: 0.9854 - val_loss: 0.0187 - val_accuracy: 0.9921
Epoch 5/5
202/202 [==============================] - 10s 51ms/step - loss: 0.0447 - accuracy: 0.9879 - val_loss: 0.0168 - val_accuracy: 0.9963


In [39]:
CNNpred = model.predict(test_features)

# GAN For -1 Detection

In [43]:
modelG = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(100,)),
            tf.keras.layers.Dense(units=200, activation=tf.nn.relu),
            tf.keras.layers.Dense(100,activation='relu'),
            tf.keras.layers.Dense(n_outputs, activation='softmax')
        ]
    )
modelD = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(n_outputs,)),
            tf.keras.layers.Dense(200, activation='relu'),
            tf.keras.layers.Dense(100,activation='relu'),
            tf.keras.layers.Dense(1,activation='sigmoid'),
        ]
    )

In [45]:
modelD.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 200)               243000    
_________________________________________________________________
dense_26 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 101       
Total params: 263,201
Trainable params: 263,201
Non-trainable params: 0
_________________________________________________________________


In [46]:
optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)

# compile the discriminator
modelD.compile(loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

# The generator takes noise as input and generates imgs
z = tf.keras.Input(shape=(100,))
img = modelG(z)

# The discriminator takes generated images as input and determines if real or fake
real = modelD(img)

combined = tf.keras.models.Model(z, real)
combined.compile(loss='binary_crossentropy', optimizer='adam')

In [97]:
batch_size=50

# Adversarial ground truths
real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

In [48]:
pred = model.predict(features)

In [ ]:
for epoch in range(500):
    #make discriminator trainable
    if (epoch % 166 == 0):
      choices = np.random.choice(8306,8300,replace=False)
    
    for layer in modelD.layers:
      layer.trainable = True
    for i in range(4):
      # Select random batch of real images
      dna = pred[choices[(epoch%166)*50:(epoch%166+1)*50]]
      # Sample noise and generate a batch of fake images
      noise = np.random.normal(0, 1, (batch_size, 100))
      gen_dna = modelG.predict(noise)

      randDNA = np.random.normal(0,1, (batch_size,660,5))
      out = model.predict(randDNA)
      # Train the discriminator (real classified as ones and generated as zeros)
      d_loss_real = modelD.train_on_batch(dna, real)
      d_loss_fake = modelD.train_on_batch(gen_dna, fake)
      d_loss_cnn = modelD.train_on_batch(out, fake)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    print ("%d [D loss: %f, acc.: %.2f%%]" % (epoch, d_loss[0], 100*d_loss[1]))
    # Train the generator (discriminator cannot be trainable)
    for layer in modelD.layers:
      layer.trainable = False
    for i in range (4):
      noise = np.random.normal(0, 1, (batch_size, 100))
      g_loss = combined.train_on_batch(noise, real)
    print ("G loss: %f" % g_loss)

Test GAN on Generated prediction

In [123]:
rand = np.random.normal(0, 1, (1, 100))
psuedoDNA = modelG.predict(rand)
print(psuedoDNA)
print(modelD.predict(psuedoDNA))

[[1.0242099e-16 1.1180212e-16 7.7807324e-16 ... 3.3492722e-17
  5.5504960e-13 3.7416605e-17]]
[[0.18541741]]


Test GAN on Noise run through CNN

In [124]:
randDNA = np.random.normal(0,1, (1,660,5))
out = model.predict(randDNA)
modelD.predict(out)

array([[0.06098208]], dtype=float32)

In [125]:
est = modelD.predict(model.predict(features))

In [128]:
pred_test = model.predict(test_features)
GANpredictions = modelD.predict(pred_test)

In [129]:
sorted(GANpredictions)[100]

array([0.07805833], dtype=float32)

In [52]:
pred = model.predict(test_features)
id = np.asarray(range(1,len(pred)+1))
count = 0
condensed = [-1]*len(pred)
for i in range(len(pred)):
  condensed[i] = list(pred[i]).index(max(pred[i]))
  if max(pred[i]) < .4:
    count += 1
    condensed[i] = -1
condensed = np.asarray(condensed)
print(count)
out = np.vstack((id,condensed)).T
np.savetxt("/content/drive/My Drive/Colab Notebooks/Kaggle Competition/predictions.csv",out,delimiter=',',fmt='%d')

46


# GAN Model To Find New Species
This GAN is similar to the previous one, but it is trained on the DNA data rather than the predictions

In [8]:
modelG = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(100,)),
            tf.keras.layers.Dense(units=165*32, activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape=(165, 32)),
            tf.keras.layers.Conv1DTranspose(
                filters=32, kernel_size=5, strides = 2, padding='same',
                activation='relu'),
            tf.keras.layers.Conv1DTranspose(
                filters=16, kernel_size=5, strides = 2, padding='same',
                activation='relu'),
            # No activation
            tf.keras.layers.Conv1DTranspose(
                filters=5, kernel_size=5, padding='same'),
        ]
    )
modelD = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(660,5)),
            tf.keras.layers.Conv1D(
                filters=16, kernel_size=5, activation='relu'),
            tf.keras.layers.Conv1D(
                filters=32, kernel_size=5, activation='relu'),
            tf.keras.layers.Flatten(),
            # No activation
            tf.keras.layers.Dense(1,activation='sigmoid'),
        ]
    )

In [9]:
modelD.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 656, 16)           416       
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 652, 32)           2592      
_________________________________________________________________
flatten_1 (Flatten)          (None, 20864)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 20865     
Total params: 23,873
Trainable params: 23,873
Non-trainable params: 0
_________________________________________________________________


In [10]:
optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)

# compile the discriminator
modelD.compile(loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

# The generator takes noise as input and generates imgs
z = tf.keras.Input(shape=(100,))
img = modelG(z)

# The discriminator takes generated images as input and determines if real or fake
real = modelD(img)

combined = tf.keras.models.Model(z, real)
combined.compile(loss='binary_crossentropy', optimizer='adam')

In [11]:
batch_size=128

# Adversarial ground truths
real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

In [ ]:
for epoch in range(1000):
    #make discriminator trainable
    for layer in modelD.layers:
      layer.trainable = True
    
    for i in range(4):
    # Select random batch of real images
      idx = np.random.randint(0, features.shape[0], batch_size)
      dna = features[idx]
      # Sample noise and generate a batch of fake images
      noise = np.random.normal(0, 1, (batch_size, 100))
      gen_dna = modelG.predict(noise)
      # Train the discriminator (real classified as ones and generated as zeros)
      d_loss_real = modelD.train_on_batch(dna, real)
      d_loss_fake = modelD.train_on_batch(gen_dna, fake)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    print ("%d [D loss: %f, acc.: %.2f%%]" % (epoch, d_loss[0], 100*d_loss[1]))
    
    # Train the generator (discriminator cannot be trainable)
    for layer in modelD.layers:
      layer.trainable = False
    for i in range (4):
      noise = np.random.normal(0, 1, (batch_size, 100))
      g_loss = combined.train_on_batch(noise, real)
    print ("G loss: %f" % g_loss)

In [13]:
#test GAN on noise
randDNA = np.random.normal(0,1, (1,660,5))
modelD.predict(randDNA)

array([[0.]], dtype=float32)

In [14]:
rand = np.random.normal(0, 1, (1, 100))
psuedoDNA = modelG.predict(rand)
print(psuedoDNA)
print(modelD.predict(psuedoDNA))

[[[ 0.03492256  0.11380237  0.10538694  0.12676129  0.01984935]
  [ 0.07650568  0.14554189  0.06029651  0.0966088   0.20774454]
  [ 0.02081677  0.14120835 -0.08018703  0.06806859  0.6372641 ]
  ...
  [-0.6873522  -0.12492496  1.2300451   0.44943583  0.81204826]
  [-1.9546212  -0.8767197   3.896583   -0.04179324  3.5478032 ]
  [-0.3235383  -0.9957245   1.2098037   0.18126544  2.9916213 ]]]
[[1.]]


Make predictions With GAN and CNN

In [15]:
GANpredictions = modelD.predict(test_features)
min(GANpredictions)

array([0.86442435], dtype=float32)

In [29]:
pred = model.predict(test_features)

id = np.asarray(range(1,len(pred)+1))

count = 0
condensed = [-1]*len(pred)
for i in range(len(pred)):
  condensed[i] = list(pred[i]).index(max(pred[i]))
  if GANpredictions[i] != 1:
    count += 1
    condensed[i] = -1
condensed = np.asarray(condensed)
print(count)
out = np.vstack((id,condensed)).T
np.savetxt("/content/drive/My Drive/Colab Notebooks/Kaggle Competition/predictions.csv",out,delimiter=',',fmt='%d')

49


#RNN Implementation of DNA Classification

In [41]:
features_df = read_csv('/content/drive/My Drive/Colab Notebooks/Kaggle Competition/train_features.csv')
labels_df = read_csv('/content/drive/My Drive/Colab Notebooks/Kaggle Competition/train_labels.csv')
features = features_df.values[:,1]
for i in range(len(features)):
  temp = np.zeros((60,10))
  # splitting into k-mers of size 10
  for j in range(len(temp)):
      for k in range(5):
        try:
          if features[i][j*10+k] == 'A':
            temp[j][k] = 1
          elif features[i][j*10+k] == 'C':
            temp[j][k] = 2
          elif features[i][j*10+k] == 'G':
            temp[j][k] = 3
          elif features[i][j*10+k] == 'T':
            temp[j][k] = 4
          else:
            temp[j][k] = 0
        except IndexError:
          temp[j][k] = 0
  features[i] = temp
features = np.stack(features)
labels = labels_df.values
trainy = tf.keras.utils.to_categorical(labels[:,1])

In [42]:
n_timesteps, k_mer_len = features.shape[1], features.shape[2]

In [43]:
features.shape

(12906, 60, 10)

In [44]:
modelRNN = tf.keras.Sequential()
modelRNN.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(10,return_sequences=True,dropout=.125),input_shape=(n_timesteps, k_mer_len)))
modelRNN.add(tf.keras.layers.Flatten())
modelRNN.add(tf.keras.layers.Dropout(.125))
modelRNN.add(tf.keras.layers.Dense(64, activation='relu'))
modelRNN.add(tf.keras.layers.Dense(n_outputs,activation='softmax'))
modelRNN.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_6 (Bidirection (None, 60, 20)            1680      
_________________________________________________________________
flatten_8 (Flatten)          (None, 1200)              0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 1200)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                76864     
_________________________________________________________________
dense_17 (Dense)             (None, 1214)              78910     
Total params: 157,454
Trainable params: 157,454
Non-trainable params: 0
_________________________________________________________________


In [45]:
modelRNN.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = modelRNN.fit(features, trainy, batch_size=64, epochs=15)

Epoch 1/15
202/202 [==============================] - 16s 41ms/step - loss: 6.9473 - accuracy: 0.0149
Epoch 2/15
202/202 [==============================] - 8s 40ms/step - loss: 5.9361 - accuracy: 0.0855
Epoch 3/15
202/202 [==============================] - 8s 41ms/step - loss: 3.5848 - accuracy: 0.3507
Epoch 4/15
202/202 [==============================] - 8s 40ms/step - loss: 1.8292 - accuracy: 0.6391
Epoch 5/15
202/202 [==============================] - 8s 41ms/step - loss: 1.0648 - accuracy: 0.7889
Epoch 6/15
202/202 [==============================] - 8s 40ms/step - loss: 0.7135 - accuracy: 0.8598
Epoch 7/15
202/202 [==============================] - 8s 40ms/step - loss: 0.5359 - accuracy: 0.8917
Epoch 8/15
202/202 [==============================] - 8s 41ms/step - loss: 0.4291 - accuracy: 0.9097
Epoch 9/15
202/202 [==============================] - 8s 40ms/step - loss: 0.3607 - accuracy: 0.9222
Epoch 10/15
202/202 [==============================] - 8s 41ms/step - loss: 0.3074 - accur

In [47]:
test_features_df = read_csv('/content/drive/My Drive/Colab Notebooks/Kaggle Competition/test_features.csv')
test_features = test_features_df.values[:,1]
for i in range(len(test_features)):
  temp = np.zeros((60,10))
  # splitting into k-mers of size 10
  for j in range(len(temp)):
      for k in range(5):
        try:
          if test_features[i][j*10+k] == 'A':
            temp[j][k] = 1
          elif test_features[i][j*10+k] == 'C':
            temp[j][k] = 2
          elif test_features[i][j*10+k] == 'G':
            temp[j][k] = 3
          elif test_features[i][j*10+k] == 'T':
            temp[j][k] = 4
          else:
            temp[j][k] = 0
        except IndexError:
          temp[j][k] = 0
  test_features[i] = temp
test_features = np.stack(test_features)

In [48]:
RNN_predictions = modelRNN.predict(test_features)

# Embedding Classification Model

In [33]:
features_df = read_csv('/content/drive/My Drive/Colab Notebooks/Kaggle Competition/train_features.csv')
labels_df = read_csv('/content/drive/My Drive/Colab Notebooks/Kaggle Competition/train_labels.csv')
features = features_df.values[:,1]
for i in range(len(features)):
  temp = np.zeros(660)
  for j in range(len(temp)):
    try:
      if features[i][j] == 'A':
        temp[j] = 1
      elif features[i][j] == 'C':
        temp[j] = 2
      elif features[i][j] == 'G':
        temp[j] = 3
      elif features[i][j] == 'T':
        temp[j] = 4
      else:
        temp[j] = 0
    except IndexError:
      temp[j] = 0
  features[i] = temp
features = np.stack(features)
labels = labels_df.values
trainy = tf.keras.utils.to_categorical(labels[:,1])

In [34]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=5, input_length=660, output_dim=32))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(.125))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(n_outputs,activation='sigmoid'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 660, 32)           160       
_________________________________________________________________
flatten_3 (Flatten)          (None, 21120)             0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 21120)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                1351744   
_________________________________________________________________
dense_7 (Dense)              (None, 1214)              78910     
Total params: 1,430,814
Trainable params: 1,430,814
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(features[:10000], trainy[:10000], validation_data=(features[10000:],trainy[10000:]),batch_size=64, epochs=10)

Epoch 1/10
157/157 [==============================] - 7s 41ms/step - loss: 6.6112 - accuracy: 0.0461 - val_loss: 5.7595 - val_accuracy: 0.1108
Epoch 2/10
157/157 [==============================] - 6s 39ms/step - loss: 3.5917 - accuracy: 0.3983 - val_loss: 1.9462 - val_accuracy: 0.6511
Epoch 3/10
157/157 [==============================] - 6s 39ms/step - loss: 0.7209 - accuracy: 0.8585 - val_loss: 0.7072 - val_accuracy: 0.8844
Epoch 4/10
157/157 [==============================] - 6s 39ms/step - loss: 0.2494 - accuracy: 0.9451 - val_loss: 0.5571 - val_accuracy: 0.9174
Epoch 5/10
157/157 [==============================] - 6s 40ms/step - loss: 0.1363 - accuracy: 0.9665 - val_loss: 0.5178 - val_accuracy: 0.9274
Epoch 6/10
157/157 [==============================] - 6s 39ms/step - loss: 0.0841 - accuracy: 0.9793 - val_loss: 0.4874 - val_accuracy: 0.9401
Epoch 7/10
157/157 [==============================] - 6s 39ms/step - loss: 0.0637 - accuracy: 0.9843 - val_loss: 0.4874 - val_accuracy: 0.9418

# Variational Auto Encoder

In [ ]:
features_df = read_csv('/content/drive/My Drive/Colab Notebooks/Kaggle Competition/train_features.csv')
labels_df = read_csv('/content/drive/My Drive/Colab Notebooks/Kaggle Competition/train_labels.csv')
features = features_df.values[:,1]
for i in range(len(features)):
  temp = np.zeros(660)
  for j in range(len(temp)):
    try:
      if features[i][j] == 'A':
        temp[j] = 1
      elif features[i][j] == 'C':
        temp[j] = 2
      elif features[i][j] == 'G':
        temp[j] = 3
      elif features[i][j] == 'T':
        temp[j] = 4
      else:
        temp[j] = 0
    except IndexError:
      temp[j] = 0
  features[i] = temp
  features[i] = tf.keras.utils.to_categorical(features[i])
features = np.stack(features)
labels = labels_df.values
trainy = tf.keras.utils.to_categorical(labels[:,1])

In [ ]:
class CVAE(tf.keras.Model):
  """Convolutional variational autoencoder."""

  def __init__(self, latent_dim):
    super(CVAE, self).__init__()
    self.latent_dim = latent_dim
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(660,5)),
            tf.keras.layers.Conv1D(
                filters=16, kernel_size=3, strides=2, activation='relu'),
            tf.keras.layers.Conv1D(
                filters=32, kernel_size=3, strides=2, activation='relu'),
            tf.keras.layers.Flatten(),
            # No activation
            tf.keras.layers.Dense(latent_dim + latent_dim),
        ]
    )
    print(self.encoder.summary())
    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
            tf.keras.layers.Dense(units=165*32, activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape=(165, 32)),
            tf.keras.layers.Conv1DTranspose(
                filters=32, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            tf.keras.layers.Conv1DTranspose(
                filters=16, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            # No activation
            tf.keras.layers.Conv1DTranspose(
                filters=5, kernel_size=3, padding='same'),
        ]
    )
    print(self.decoder.summary())

  @tf.function
  def sample(self, eps=None):
    if eps is None:
      eps = tf.random.normal(shape=(100, self.latent_dim))
    return self.decode(eps, apply_sigmoid=True)

  def encode(self, x):
    mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
    return mean, logvar

  def reparameterize(self, mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean

  def decode(self, z, apply_sigmoid=False):
    logits = self.decoder(z)
    if apply_sigmoid:
      probs = tf.sigmoid(logits)
      return probs
    return logits

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-4)


def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis)


def compute_loss(model, x):
  mean, logvar = model.encode(x)
  z = model.reparameterize(mean, logvar)
  x_logit = model.decode(z)
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
  logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2])
  logpz = log_normal_pdf(z, 0., 0.)
  logqz_x = log_normal_pdf(z, mean, logvar)
  return -tf.reduce_mean(logpx_z + logpz - logqz_x)


@tf.function
def train_step(model, x, optimizer):
  """Executes one training step and returns the loss.

  This function computes the loss and gradients, and uses the latter to
  update the model's parameters.
  """
  with tf.GradientTape() as tape:
    loss = compute_loss(model, x)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [ ]:
epochs = 7
# set the dimensionality of the latent space to a plane for visualization later
latent_dim = 50

model = CVAE(latent_dim)

In [ ]:
for epoch in range(1, epochs + 1):
  start_time = time.time()
  for train_x in range(len(features)):
    train_step(model, features[train_x:train_x+1], optimizer)
  end_time = time.time()

  loss = tf.keras.metrics.Mean()
  for test_x in range(len(features)):
    loss(compute_loss(model, features[test_x:test_x+1]))
  elbo = -loss.result()
  #display.clear_output(wait=False)
  print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
        .format(epoch, elbo, end_time - start_time))

In [ ]:
latent_features = np.zeros((len(features),latent_dim))
for i in range(len(features)):
  mean, logvar = model.encode(features[i:i+1])
  z = model.reparameterize(mean,logvar)
  z = z.numpy()[0]
  latent_features[i] = z

In [ ]:
classifier = tf.keras.Sequential()
classifier.add(tf.keras.layers.Dense(64,activation='relu',input_shape=(50,)))
classifier.add(tf.keras.layers.Dense(100,activation='relu'))
classifier.add(tf.keras.layers.Dense(1214,activation='sigmoid'))
classifier.summary()

In [ ]:
classifier.compile(optimizer=keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
classifier.fit(latent_features[:11000], trainy[:11000], epochs=15, validation_data = (latent_features[11000:],trainy[11000:]),batch_size=64)

In [ ]:
test_features_df = read_csv('/content/drive/My Drive/Colab Notebooks/Kaggle Competition/test_features.csv')
test_features = test_features_df.values[:,1]
for i in range(len(test_features)):
  temp = np.zeros(660)
  for j in range(len(temp)):
    try:
      if test_features[i][j] == 'A':
        temp[j] = 1
      elif test_features[i][j] == 'C':
        temp[j] = 2
      elif test_features[i][j] == 'G':
        temp[j] = 3
      elif test_features[i][j] == 'T':
        temp[j] = 4
      else:
        temp[j] = 0
    except IndexError:
      temp[j] = 0
  test_features[i] = temp
  test_features[i] = tf.keras.utils.to_categorical(test_features[i])
test_features = np.stack(test_features)

In [ ]:
latent_test_features = np.zeros((len(test_features),latent_dim))
for i in range(len(test_features)):
  mean, logvar = model.encode(test_features[i:i+1])
  z = model.reparameterize(mean,logvar)
  z = z.numpy()[0]
  latent_test_features[i] = z

In [ ]:
pred = classifier.predict(latent_test_features)
id = np.asarray(range(1,len(pred)+1))

count = 0
condensed = [-1]*len(pred)
for i in range(len(pred)):
  condensed[i] = list(pred[i]).index(max(pred[i]))
condensed = np.asarray(condensed)
print(count)
out = np.vstack((id,condensed)).T
np.savetxt("/content/drive/My Drive/Colab Notebooks/Kaggle Competition/predictions.csv",out,delimiter=',',fmt='%d')

# Word2Vec
This is not working yet. It requires work on sampling and context generation

Code from: https://www.tensorflow.org/tutorials/text/word2vec

In [27]:
from itertools import product
indexes = list(product([0,1, 2,3,4], repeat=3))

In [28]:
features_df = read_csv('/content/drive/My Drive/Colab Notebooks/Kaggle Competition/train_features.csv')
labels_df = read_csv('/content/drive/My Drive/Colab Notebooks/Kaggle Competition/train_labels.csv')
features = features_df.values[:,1]
for i in range(len(features)):
  temp = np.zeros((200,1))
  # splitting into k-mers of size 10
  for j in range(len(temp)):
    t = np.zeros(3,dtype=int)
    for k in range(3):
      try:
        if features[i][j*3+k] == 'A':
          t[k] = 1
        elif features[i][j*3+k] == 'C':
          t[k] = 2
        elif features[i][j*3+k] == 'G':
          t[k] = 3
        elif features[i][j*3+k] == 'T':
          t[k] = 4
        else:
          t[k] = 0
      except IndexError:
        t[k] = 0
    temp[j] = indexes.index(tuple(t))
  features[i] = temp
features = np.stack(features)
labels = labels_df.values
trainy = tf.keras.utils.to_categorical(labels[:,1])

In [46]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = tf.keras.layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = tf.keras.layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=5+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [47]:
embedding_dim = 128
word2vec = Word2Vec(len(indexes), embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [41]:
features.shape

(12906, 200, 1)